In [1]:
import pandas as pd
train_data = pd.read_csv("train.tsv",sep = "\t" ,header = 0)
test_data = pd.read_csv("test.tsv", sep ="\t" , header = 0)

In [2]:
train_data.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [3]:
import nltk
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
def process(phrase):
    temp = []
    stem = PorterStemmer()
    words = word_tokenize(phrase)
    temp = [stem.stem(a.lower()) for a in words]
    return ' '.join(temp)
    

In [5]:
punctuations = string.punctuation
stop_words = list(stopwords.words("english"))
x = train_data["Phrase"]
train_data["Phrase"] = train_data["Phrase"].apply(lambda x: process(x))


In [86]:
tf = TfidfVectorizer()
train_x = tf.fit_transform(train_data["Phrase"])
train_y = train_data["Sentiment"]
train_x

<156060x10998 sparse matrix of type '<class 'numpy.float64'>'
	with 970485 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [9]:
multinomial_nb_classifier = MultinomialNB().fit(train_x,train_y)

In [10]:
multinomial_logistic_classifier = LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(train_x, train_y)

In [11]:
classifier_SVM = SVC(kernel = 'linear')
classifier_SVM.fit(train_x,train_y)
# classifier_random_forest = RandomForestClassifier(n_estimators=10, random_state=0)
# classifier_random_forest.fit(train_x,train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [41]:
from sklearn.model_selection import train_test_split
x_tr,x_te,y_tr,y_te = train_test_split(train_x,train_y,test_size =0.3,random_state =1)
x_te

<46818x10998 sparse matrix of type '<class 'numpy.float64'>'
	with 291309 stored elements in Compressed Sparse Row format>

In [21]:
multinomial_nb_classifier_dummy = MultinomialNB().fit(x_tr,y_tr)
multinomial_logistic_classifier_dummy = LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(x_tr, y_tr)

In [97]:
predicted_value_NB_dummy= multinomial_nb_classifier_dummy.predict(x_tr)
predicted_value_logistic_dummy = multinomial_logistic_classifier_dummy.predict(x_tr)
# predicted_value_svm = classifier_svm.predict(x_test)

In [98]:
print(metrics.accuracy_score(y_tr,predicted_value_logistic_dummy))
print(metrics.accuracy_score(y_tr,predicted_value_NB_dummy))

0.674548250673
0.61379322971


In [99]:
#we see that logistic classifier performs better ... so now 
# testing the logistic model(which was trained on whole of train dataset) on whole of test_data

# preprocessing the test_data
test_data["Phrase"] = test_data["Phrase"].apply(lambda x: process(x))

In [109]:
test_x = tf.transform(test_data["Phrase"])

In [110]:
predicted_value_NB = multinomial_nb_classifier.predict(test_x)
predicted_value_logistic = multinomial_logistic_classifier.predict(test_x)


In [111]:
print(predicted_value_NB)
print(predicted_value_logistic)

[2 2 2 ..., 2 2 2]
[2 2 2 ..., 2 2 2]


In [112]:
test_data["Sentiment"] = predicted_value_logistic
test_data[["PhraseId","Sentiment"]].head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [113]:
test_data[["PhraseId","Sentiment"]].to_csv("submission.csv", sep ="," , index =False)